In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
# set all pytorch default tensortype to DoubleTensor
torch.set_default_tensor_type('torch.DoubleTensor')
# for large neural nets, improve performance
torch.backends.cudnn.benchmark=True
# seed random seed
torch.manual_seed(21)

In [3]:
args={
    "batch_size_GPU":150,
    "batch_size_CPU":4
}

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_loader = data.DataLoader(
        datasets.ImageFolder('imagefolder',
                             transforms.Compose([
                                 transforms.Resize((150,150)),
                                 #transforms.RandomResizedCrop(224),
                                 #transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 normalize,
                             ])),
        batch_size=args["batch_size_GPU"],
        shuffle=True,
        num_workers=5,
pin_memory=True)

test_loader = data.DataLoader(
        datasets.ImageFolder('test_imagefolder',
                             transforms.Compose([
                                 transforms.Resize((150,150)),
                                 #transforms.RandomResizedCrop(224),
                                 #transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 normalize,
                             ])),
        batch_size=args["batch_size_GPU"],
        shuffle=True,
        num_workers=5,
pin_memory=True)
classes = ("cat","dog")

In [5]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()


In [6]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        #self.conv3 = nn.Conv2d(16, 34, 5)
        #self.conv4 = nn.Conv2d(26, 36, 5)
        self.fc1 = nn.Linear(16 * 34 * 34, 120)
        self.fc2 = nn.Linear(120, 80)
        self.fc3 = nn.Linear(80, 10)
        #self.fc4 = nn.Linear(120, 80)
        #self.fc5 = nn.Linear(80, 10)
        #self.fc6 = nn.Linear(45,10)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #x = self.pool(F.relu(self.conv3(x)))
        #x = self.pool(F.relu(self.conv4(x)))
        # refer to this x.size() when changing img size, layers etc
        #print(x.size())
        x = x.view(-1, 16 * 34 * 34)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        #x = F.relu(self.fc5(x))
        x = self.fc3(x)
        return x


net = Net()
net.load_state_dict(torch.load('cat-v-dog-model-120-80-10-10iter.pt'))
net.cuda()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=18496, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=80, bias=True)
  (fc3): Linear(in_features=80, out_features=10, bias=True)
)

In [7]:
dataiter = iter(test_loader)
testimages, labels = dataiter.next()
testimages = testimages.type('torch.DoubleTensor')
testimages = testimages.type('torch.cuda.DoubleTensor')
outputs = net(testimages)
_, predicted = torch.max(outputs, 1)
predicted
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Predicted:    cat   dog   cat   dog


In [8]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.type('torch.cuda.DoubleTensor')
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.type('torch.DoubleTensor')
        total += labels.size(0)
        labels = labels.type('torch.DoubleTensor')
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 73 %


In [ ]:
"""
Utility function for computing output of convolutions
takes a tuple of (h,w) and returns a tuple of (h,w)
"""
def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    h = floor( ((h_w[0] + (2 * pad) - ( dilation * (kernel_size[0] - 1) ) - 1 )/ stride) + 1)
    print("H calc")
    w = floor( ((h_w[1] + (2 * pad) - ( dilation * (kernel_size[1] - 1) ) - 1 )/ stride) + 1)
    print("w calc")
    return h, w

In [ ]:
conv_output_shape((6,16))